# Hausholder transformation project 
##### made  for numerical analysis classes by Kamil Bartocha, Marcin Baranek and Mateusz Kamyczura

### Import Libraries

In [1]:
import numpy as np
import math
import time

### Input matrix from user

In [2]:
def input_matrix():
    # Take matrix size as input
    while True:
        try:
            row_number = int(input("enter number of rows: "))
            column_number = int(input("enter number of columns: "))
            break
        except ValueError:
            print("That was no valid number. Try to input integer again ")
        
    # initialise matrix with zeroes
    matrix = np.zeros((row_number, column_number))

    # input each element row at a time
    for row_iterator in range(row_number):
        for column_iterator in range(column_number):
            while True:
                try:
                    matrix[row_iterator][column_iterator] = input(f'enter value of matrix[{row_iterator}][{column_iterator}] = ')
                    break
                except ValueError:
                    print("That was no valid number. Try to input number again ")   
    return matrix

### Householder transformation 

In [3]:
def householder_transformation_matrix(vector: np.ndarray, new_vector: np.ndarray,
                                      time_of_execution=False) -> np.ndarray:

    print(f"\ncompute transformation matrix for: {vector} innew direction: {new_vector}")
    if time_of_execution:
        now = time.time()

    vector = np.reshape(vector, newshape=(-1, 1))
    new_vector = np.reshape(new_vector, newshape=(-1, 1))

    dimensional = vector.shape[0]

    cos_of_angel_between_vector_and_new_vector = np.dot(np.transpose(vector), new_vector)
    # for numerical stability we chose a vector of more norm
    if cos_of_angel_between_vector_and_new_vector > 0:
        householder_vector = vector + math.sqrt((vector ** 2).sum()) * new_vector
    else:
        householder_vector = vector - math.sqrt((vector ** 2).sum()) * new_vector

    specular_reflection = np.dot(np.transpose(householder_vector), householder_vector)
    if specular_reflection != 0:
        specular_reflection = 2 / specular_reflection
    else:
        #vector and direction vector have the same direction so return identity matrix
        if time_of_execution:
            print(f"time of execution householder_transformation_matrix is: {time.time() - now} s")
        return np.eye(dimensional)

    identity_matrix = np.eye(dimensional)

    #create householder matrix
    householder_matrix = specular_reflection * np.dot(householder_vector, np.transpose(householder_vector))
    householder_matrix = identity_matrix - householder_matrix

    if time_of_execution:
        print(f"time of execution householder_transformation_matrix is: {time.time() - now} s")

    print(f"transformation matrix:\n"
          f"{householder_matrix}")

    return householder_matrix


### improvement r matrix
###### for numerical stability, inserts 0 where for sure they are (for upper triangular matrix)

In [4]:
def improvement_r_matrix(matrix: np.ndarray, iteration: int) -> np.ndarray:
    j = 1
    for i in range(iteration):
        matrix[j:, i] = np.zeros(shape=matrix[j:, i].shape)
        j += 1
    return matrix

### improvement matrix
###### for numerical stability, inserts $\epsilon$ where value is close to 0

In [5]:
def improvement_matrix(matrix: np.ndarray, epsilon=1.e-7):
    matrix = matrix.astype(dtype=float)
    location = np.where(abs(matrix - 0) < epsilon)
    matrix[location] = epsilon
    return matrix

### Householder algorithm

In [6]:
def householder_algorithm(matrix: np.ndarray,
                          improve_matrix=True,
                          improve_r_matrix=True,
                          time_of_execution=False) -> (np.ndarray, np.ndarray):

    print(f"\ncompute matrix decomposition for\n"
          f"{matrix}")

    if time_of_execution:
        now = time.time()

    if improve_matrix:
        matrix = improvement_matrix(matrix)

    first_dimensional = matrix.shape[0]

    if first_dimensional > matrix.shape[1]:
        print("The matrix has the first dimension greater than the second,\n\n"
              "therefore the algorithm was run for the transposed matrix")
        first_dimensional = matrix.shape[1]
        matrix = np.transpose(matrix)

    identity_matrix = np.eye(first_dimensional)

    # create P, Q and R matrix
    # Q matrix will be transpose be for end of function
    p_matrix = identity_matrix
    q_matrix = np.zeros(shape=p_matrix.shape)
    r_matrix = matrix

    # create unit vector
    unit_vector = np.zeros(shape=(first_dimensional, 1)).astype(int)
    unit_vector[0, 0] = 1

    for i in range(first_dimensional):

        householder_matrix = householder_transformation_matrix(r_matrix[i:, i], unit_vector)

        if not householder_matrix.all():
            print("something went wrong")
            print("check the assumptions")
            return False, False

        # preparing unit vector to next iteration
        if i < first_dimensional - 1:
            unit_vector = np.reshape(np.delete(unit_vector, -1), newshape=(-1, 1))

        # update P matrix
        if i < first_dimensional - 1:
            p_matrix[i:, i:] = householder_matrix

        print(f"\ncurrent P matrix:\n\n"
              f"{p_matrix}")

        # update R matrix
        r_matrix = np.dot(p_matrix, r_matrix)
        if improve_r_matrix:
            r_matrix = improvement_r_matrix(r_matrix, i)

        print(f"\ncurrent R matrix:\n\n"
              f"{r_matrix}")

        # update Q matrix
        if i == 0:
            q_matrix = p_matrix
        else:
            q_matrix = np.dot(p_matrix, q_matrix)

        print(f"\n current Q matrix:\n\n"
              f"{q_matrix}")

        # preparing P matrix to next iteration
        p_matrix = np.eye(first_dimensional)

    q_matrix = np.transpose(q_matrix)

    if time_of_execution:
        print(f"time of execution householder_algorithm is: {time.time() - now} s")

    print(f"\nfunction return Q matrix:\n\n"
          f"{q_matrix}\n\n"
          f"and R matrix:\n\n"
          f"{r_matrix}")

    return q_matrix, r_matrix


### Linear equations with using hausholder algorithm

In [7]:
def linear_equations_with_householder_algorithm(matrix: np.ndarray,
                                                bias: np.ndarray,
                                                time_of_execution=False) -> np.ndarray:

    print(f"\ncompute linear equations with matrix:\n\n {matrix} with bias: {bias}\n")

    if time_of_execution:
        now = time.time()

    first_dimensional = matrix.shape[0]

    if matrix.shape[0] != matrix.shape[1]:
        print("Error in linear_equations_with_householder_algorithm:\n\n"
              "The input matrix has wrong shape")
        return np.array(False)

    bias = np.reshape(bias, newshape=(-1, 1))

    if bias.shape[0] != first_dimensional:
        print("Error in linear_equations_with_householder_algorithm:\n\n"
              "the bias has the wrong shape")
        return np.array(False)

    q_matrix, r_matrix = householder_algorithm(matrix, improve_matrix=True, improve_r_matrix=True)
    bias = np.dot(np.transpose(q_matrix), bias)
    solve = np.array(False)

    for i in range(1, first_dimensional + 1, 1):
        if r_matrix[-i, - i] == 0:
            print("Error in linear_equations_with_householder_algorithm:\n\n"
                  "the system is contradictory")
            return np.array(False)

        if i == 1:
            solve = np.array(bias[-i, 0] / r_matrix[-i, -i])
        else:
            next_solve_value = (bias[-i, 0] + np.dot(r_matrix[- i, 1 - i:], solve)) / r_matrix[- i, - i]
            solve = np.concatenate((np.reshape(next_solve_value, newshape=(-1, 1)),
                                    np.reshape(solve, newshape=(-1, 1))), axis=0)
            solve = np.reshape(solve, newshape=(-1, 1))

        print(f"\ncurrent solve: {solve}")

    if time_of_execution:
        print(f"\ntime of execution linear_equations_with_householder_algorithm is: {time.time() - now} s")

    return solve

In [8]:
def ms_erorr(matrix_1: np.ndarray, matrix_2: np.ndarray):
    return ((matrix_1 - matrix_2) **2).sum()

### Execution 

In [9]:
print("Input matrix n x n : ")
a_matrix = np.array(input_matrix())
print("Input bias 1 x n: ")
bias = np.array(input_matrix())

q_matrix, r_matrix = householder_algorithm(a_matrix)
erorr = ms_erorr(a_matrix, np.dot(q_matrix, r_matrix))
linear_equations_with_householder_algorithm(a_matrix, bias)
print(f"MSE between a_matrix and q_matrix multiplay by r_matrix is equal:  {erorr}")

Input matrix n x n : 
enter number of rows: 3
enter number of columns: 3
enter value of matrix[0][0] = 1
enter value of matrix[0][1] = 2
enter value of matrix[0][2] = 3
enter value of matrix[1][0] = 4
enter value of matrix[1][1] = 5
enter value of matrix[1][2] = 6
enter value of matrix[2][0] = 7
enter value of matrix[2][1] = 8
enter value of matrix[2][2] = 9
Input bias 1 x n: 
enter number of rows: 1
enter number of columns: 3
enter value of matrix[0][0] = 1
enter value of matrix[0][1] = 2
enter value of matrix[0][2] = 3

compute matrix decomposition for
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

compute transformation matrix for: [1. 4. 7.] innew direction: [[1]
 [0]
 [0]]
transformation matrix:
[[-0.12309149 -0.49236596 -0.86164044]
 [-0.49236596  0.7841456  -0.3777452 ]
 [-0.86164044 -0.3777452   0.33894589]]

current P matrix:

[[-0.12309149 -0.49236596 -0.86164044]
 [-0.49236596  0.7841456  -0.3777452 ]
 [-0.86164044 -0.3777452   0.33894589]]

current R matrix:

[[-8.12403840e+00 -9.60